### Overview about the project:
In this project I will overview the dataset 525-bird-species. I will explore the data, transform it into a format for modeling, and then create classification models to predict the species of birds given a picture of a bird.

### About The Data:

Data set of 525 bird species. 84635 training images, 2625 test images and 2625 validation images
All images are 224 X 224 X 3 color images in jpg format. Data set includes a train set, test set and validation set. Each set contains 525 sub directories, one for each bird species

### Summary Of Methods:

For each model, I used the imagedatagenerator in keras. I created each model as described below. Each model is compiled using a batchsize/samples ratio for the epochs_per_step (on training and validation steps). The compiled models are fit to the generator data for epochs required with patience of 5. I graphed the validation and training data loss and accuracy as a function of epochs. Finally, the third dataset - test - was compared to the model to give a true accuracy and loss value for each fit model.

### Analysis of Results:
Best accuracy achived by MobileNet(94.5% accuracy)

In [1]:
#Import packages used here:

import numpy as np
import os
import random
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator

print(np.__version__)

1.24.3


In [2]:
# setup my directories
train_directory = 'input/100-bird-species/train'
test_directory = 'input/100-bird-species/test'
val_directory = 'input/100-bird-species/valid'

In [3]:
categories = os.listdir(train_directory) # list the names of the categories that in my data
print(str(len(categories)),'CATEGORIES are ', categories)

category_count = len(categories)
# we will use "category_count" as the number of classes that we need to classify instead of typing 525 because the data is continuously increasing

525 CATEGORIES are  ['ABBOTTS BABBLER', 'ABBOTTS BOOBY', 'ABYSSINIAN GROUND HORNBILL', 'AFRICAN CROWNED CRANE', 'AFRICAN EMERALD CUCKOO', 'AFRICAN FIREFINCH', 'AFRICAN OYSTER CATCHER', 'AFRICAN PIED HORNBILL', 'AFRICAN PYGMY GOOSE', 'ALBATROSS', 'ALBERTS TOWHEE', 'ALEXANDRINE PARAKEET', 'ALPINE CHOUGH', 'ALTAMIRA YELLOWTHROAT', 'AMERICAN AVOCET', 'AMERICAN BITTERN', 'AMERICAN COOT', 'AMERICAN DIPPER', 'AMERICAN FLAMINGO', 'AMERICAN GOLDFINCH', 'AMERICAN KESTREL', 'AMERICAN PIPIT', 'AMERICAN REDSTART', 'AMERICAN ROBIN', 'AMERICAN WIGEON', 'AMETHYST WOODSTAR', 'ANDEAN GOOSE', 'ANDEAN LAPWING', 'ANDEAN SISKIN', 'ANHINGA', 'ANIANIAU', 'ANNAS HUMMINGBIRD', 'ANTBIRD', 'ANTILLEAN EUPHONIA', 'APAPANE', 'APOSTLEBIRD', 'ARARIPE MANAKIN', 'ASHY STORM PETREL', 'ASHY THRUSHBIRD', 'ASIAN CRESTED IBIS', 'ASIAN DOLLARD BIRD', 'ASIAN GREEN BEE EATER', 'ASIAN OPENBILL STORK', 'AUCKLAND SHAQ', 'AUSTRAL CANASTERO', 'AUSTRALASIAN FIGBIRD', 'AVADAVAT', 'AZARAS SPINETAIL', 'AZURE BREASTED PITTA', 'AZURE JAY'

In [4]:
augmented_gen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest') 

general_datagen = ImageDataGenerator(rescale = 1./255) # for training, validation and testing data

train_generator = general_datagen.flow_from_directory(
    train_directory,
    target_size = (224, 224),
    batch_size = 32
)
valid_generator = general_datagen.flow_from_directory(
    val_directory,
    target_size = (224, 224),
    batch_size = 32
)
test_generator = general_datagen.flow_from_directory(
    test_directory,
    target_size = (224, 224),
    batch_size = 32
)

Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


In [5]:
# ploting a sample from my data
def plot_image(generator):
    images_in_batch = next(generator) # images_in_batch will output (batch_size, height, width, n_channels)
    img = images_in_batch[0][0] # img will output (height, width, n_chennels)
    
    plt.imshow(img)

# plot_image(train_generator)

In [6]:
# i will be using it to determine steps_per_epoch in my models.
train_groups = len(train_generator) 
valid_groups = len(valid_generator) # validation_step

print(f"Train groups: {train_groups}")
print(f"Validation groups: {valid_groups}")

Train groups: 2645
Validation groups: 83


## Playing with my base model

In [7]:
# convolution layer that we will be used in out model i added batch normalization to accelerate the model and for generalization
def conv_layer(inputs, filters, kernel_size=3, padding="valid"):
    x = layers.Conv2D(filters = filters, kernel_size = kernel_size, padding = padding, use_bias = False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    return x
# pooling layer i added dropout cause it help my model to reduce the overfitting
def pooling_layer(inputs, pool_size = 2, dropout_rate=0.5):
    x = layers.MaxPooling2D(pool_size = pool_size)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)
    
    return x

# this dense layer i will not only use it for my base model i will use it in the pretrained model too
def dense_layer(inputs, out, dropout_rate = 0.5):
    x = layers.Dense(out)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    
    return x

In [8]:
# in first my model was only two blocks gradually from 64 to 128 filters and gives me an accuracy of 55%
# so i added the dropouts and the batch normalization that above and set my filters to 64 and to make it better i added another block
# and changes in the dense layer before my output and play with my optimizer, the model gives me an acc of +75& on the test data
# i stoped then, with more to add to my arch to make it better, i can add residual connection to it and play with things but i stopped cause it take time and the gpu

keras.backend.clear_session()

inputs = keras.Input(shape = (224, 224, 3))

x = conv_layer(inputs, 64, padding = "same")  # 224x224
x = conv_layer(x, 64)                         # 222x222
x = pooling_layer(x)                          # 111x111

x = conv_layer(x, 64, padding = "same")       # 111x111
x = conv_layer(x, 64)                         # 109x109
x = pooling_layer(x)                          # 54x54

x = conv_layer(x, 64, padding = "same")       # 54x54
x = conv_layer(x, 64)                         # 52X52
x = pooling_layer(x)                          # 26x26

x = conv_layer(x, 64, padding = "same")       # 26x26

x = layers.Flatten()(x)                       # 26*26*64

x = dense_layer(x, 512)

outputs = layers.Dense(category_count, activation = "softmax")(x)

base_model = keras.Model(inputs, outputs)

base_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1728      
                                                                 
 batch_normalization (Batch  (None, 224, 224, 64)      256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 224, 224, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 222, 222, 64)      36864     
                                                                 
 batch_normalization_1 (Bat  (None, 222, 222, 64)      256       
 chNormalization)                                            

In [ ]:
base_model.compile(optimizer =keras.optimizers.Adam(learning_rate = 0.001),
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])
#fit model
history = base_model.fit( 
    train_generator,
    steps_per_epoch = train_groups,
    epochs = 20, # adding more epochs will increase the acc like 1% or 2%
    validation_data = valid_generator,
    validation_steps = valid_groups,
    verbose = 1,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights = True),
               keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.7, patience = 2, verbose = 1),
    keras.callbacks.ModelCheckpoint(
            filepath = "intial_model.keras",
            save_best_only = True,
            monitor = "val_loss")
    ])

Epoch 1/20
   1/2645 [..............................] - ETA: 9:26:52 - loss: 6.7041 - accuracy: 0.0000e+00

In [ ]:
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(accuracy) + 1)

plt.plot(epochs, accuracy, "bo", label = "Trianing accuracy")
plt.plot(epochs, val_accuracy, "b-", label = "Validation accuracy")
plt.title("Accuracy on training and validation data")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "bo", label = "Trianing loss")
plt.plot(epochs, val_loss, "b-", label = "Validation loss")
plt.title("loss on training and validation data")
plt.title("loss on training and validation data")
plt.legend()
plt.show()

In [ ]:
# testing the model on the testset
test_model_base = keras.models.load_model("intial_model.keras")
_, test_acc = test_model_base.evaluate(test_generator)
print(f"The accuracy of the intial model on the test set is : {test_acc:.3f}")

## Transfer learning

#### we will use transfer learnig as feature extraction we will use some models, i used (VGG16 & mobilenet and (Xception"i delete it cause its performance like mobilenet in acc and it take longer time time"))

### The VGG16 model

In [ ]:
# first it gives me an acc of 75% and after some changes with the dense layer and the callbacks it gives me an acc of almost 80% on my test data
# but the loss i didn't love it, it get higher and make much changes but it didn't get well!
keras.backend.clear_session()

# load the model
conv_base_vgg16 = keras.applications.vgg16.VGG16(
    include_top = False,
    weights = "imagenet",
    input_shape = (224, 224, 3)
)
conv_base_vgg16.trainable = False # beacuse we will use it as a feature extraction

inputs = keras.Input(shape = (224, 224, 3))

x = conv_base_vgg16(inputs)

x = layers.Flatten()(x)

x = dense_layer(x, 512)

outputs = layers.Dense(category_count, activation = "softmax")(x)
vgg16_model = keras.Model(inputs, outputs)

vgg16_model.summary()

In [ ]:
vgg16_model.compile(optimizer = tf.keras.optimizers.Adam(lr=0.001),
                    loss = "categorical_crossentropy",
                    metrics = ["accuracy"]
)
callbacks=[
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights = True),
    
    keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.7, patience = 2, verbose = 1),
    
    keras.callbacks.ModelCheckpoint( filepath = "vgg16_model.keras",save_best_only = True,monitor = "val_loss")
]

history = vgg16_model.fit( 
    train_generator, 
    steps_per_epoch = train_groups, 
    epochs = 20, # adding more epochs will increase the acc like 1% or 2%
    validation_data = valid_generator,
    validation_steps = valid_groups,
    verbose = 1,
    callbacks=callbacks
)

In [ ]:
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(accuracy) + 1)

plt.plot(epochs, accuracy, "bo", label = "Trianing accuracy")
plt.plot(epochs, val_accuracy, "b-", label = "Validation accuracy")
plt.title("Accuracy on training and validation data")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "bo", label = "Trianing loss")
plt.plot(epochs, val_loss, "b-", label = "Validation loss")
plt.title("loss on training and validation data")
plt.title("loss on training and validation data")
plt.legend()
plt.show()

In [ ]:
test_model_vgg16 = keras.models.load_model("vgg16_model.keras")
_, test_acc = test_model_vgg16.evaluate(test_generator)
print(f"The accuracy of the vgg16 model on the test set is : {test_acc:.3f}")

### MobileNet

In [ ]:
keras.backend.clear_session()

base_mobilenet = keras.applications.MobileNet(
    weights = 'imagenet',
    include_top = False, 
    input_shape = (224, 224, 3)
)
base_mobilenet.trainable = False

inputs = keras.Input(shape = (224, 224, 3))

x = base_mobilenet(inputs)

x = layers.Flatten()(x)

x = dense_layer(x, 512)

outputs = layers.Dense(category_count, activation = "softmax")(x)
mobilenet_model = keras.Model(inputs, outputs)

In [ ]:
mobilenet_model.compile(optimizer = tf.keras.optimizers.Adam(lr=0.001),
                        loss = "categorical_crossentropy",
                        metrics = ["accuracy"]
)
callbacks=[
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights = True),
    
    keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.7, patience = 2, verbose = 1),
    
    keras.callbacks.ModelCheckpoint( filepath = "mobilenet_model.keras",save_best_only = True,monitor = "val_loss")
]
history = mobilenet_model.fit( 
    train_generator, 
    steps_per_epoch = train_groups, 
    epochs = 20, # adding more epochs will increase the acc like 1% or 2%
    validation_data = valid_generator,
    validation_steps = valid_groups,
    verbose = 1,
    callbacks=callbacks
)

In [ ]:
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(accuracy) + 1)

plt.plot(epochs, accuracy, "bo", label = "Trianing accuracy")
plt.plot(epochs, val_accuracy, "b-", label = "Validation accuracy")
plt.title("Accuracy on training and validation data")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "bo", label = "Trianing loss")
plt.plot(epochs, val_loss, "b-", label = "Validation loss")
plt.title("loss on training and validation data")
plt.title("loss on training and validation data")
plt.legend()
plt.show()

In [ ]:
test_model_mobile = keras.models.load_model("mobilenet_model.keras")
_, test_acc = test_model_mobile.evaluate(test_generator)
print(f"The accuracy of the mobilenet model on the test set is : {test_acc:.3f}")